In [ ]:
# Setup for Google Colab (optional)
import sys

from py4DSTEM.preprocess import swap_RQ

if 'google.colab' in sys.modules:
    print("Running in Google Colab")
    # Install required packages
    !pip install -q py4DSTEM hyperspy scikit-image matplotlib numpy scipy
    
    # Clone the repository to access data
    !git clone -q https://github.com/NU-MSE-LECTURES/465-WINTER2026.git
    import os
    os.chdir('/content/465-WINTER2026')
    
    # Set up file handling
    from google.colab import files
    print("Colab setup complete!")
else:
    print("Running in local environment")

<a href="https://colab.research.google.com/github/NU-MSE-LECTURES/465-WINTER2026/blob/main/Week_02/assignments/assignment_02_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 02: 4D-STEM Foundations

Complete this assignment to demonstrate your understanding of 4D-STEM data analysis and calibration.


In [1]:
# Colab setup
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab. Installing requirements...")
    !pip install hyperspy ase py4DSTEM
    !git clone https://github.com/NU-MSE-LECTURES/465_Computational_Microscopy_2026.git
    print("Setup complete.")
except ImportError:
    IN_COLAB = False
    print("Not running in Google Colab.")

Not running in Google Colab.


## Task 1: Distinguish Navigation vs. Signal Axes

In your notebook, define the "Navigation Axes" (where the measurement is made, e.g., x, y scan positions).

Define the "Signal Axes" (what is measured at each point, e.g., an EELS spectrum or a 2D diffraction pattern).

Use signal.axes manager to print and verify the dimensionality of a 4D-STEM dataset (expected: 2 Navigation, 2 Signal).

In [51]:
import hyperspy.api as hs
import py4DSTEM
import numpy as np
# Load the Au calibration reference dataset
# This is a small 4D-STEM dataset with known crystalline structures

filepath = '/Users/ginocangialosi/Documents/MSE 465/MATSCI465_GinoCangialosi/Week_02/assignments/raw_data/Si-SiGe.dm4'

try:
    s = py4DSTEM.io.import_file(filepath)
    print(f"✓ Dataset loaded successfully!")
    print(f"\nDataset information:")
    print(f"  Shape: {dataset_calib.shape} (order: Rx, Ry, Qx, Qy)")
    print(f"  Diffraction pattern size: {dataset_calib.shape[2:]} pixels")
    print(f"  Scan area: {dataset_calib.shape[0]} x {dataset_calib.shape[1]} pixels")
except FileNotFoundError:
    print(f"✗ File not found: {filepath}")
    print("Make sure SI_Au_calib.dm4 is in the raw_data folder")



# Navigation Axes - the scan positions
#axes_manager[0].name = 'Rx' #axis name
#axes_manager[0].units = 'x' #axis units
#axes_manager[0].scale = 1 #scale
#axes_manager[0].offset = 0 #offset

#axes_manager[1].name = 'Ry' #axis name
#axes_manager[1].units = 'y' #axis units
#axes_manager[1].scale = 0.1 #scale
#axes_manager[1].offset = 0 #offset
# Signal Axes - the measurements made at each scan position

✓ Dataset loaded successfully!

Dataset information:
  Shape: (480, 448, 77, 17) (order: Rx, Ry, Qx, Qy)
  Diffraction pattern size: (77, 17) pixels
  Scan area: 480 x 448 pixels


## Task 2: Load and Calibrate 4D-STEM Data

Use py4DSTEM.io.read to load a 4D-STEM dataset (e.g., .dm4 or .h5).

**Note:** The dataset Si-SiGe.dm4 should be available in the raw_data folder.

Set the scan step size (real space calibration) using dataset.set_scan_step_size().

Perform Center of Mass (CoM) correction using dataset.get_diffraction_shifts() to center the unscattered beam.

In [106]:
# Your code here
import py4DSTEM

#step_size = 0.1
filepath = '/Users/ginocangialosi/Documents/MSE 465/MATSCI465_GinoCangialosi/Week_02/assignments/raw_data/Si-SiGe.dm4'


dataset = py4DSTEM.io.import_file(filepath)

dataset.data

# Check the current calibration metadata
# py4DSTEM stores calibration in the DataCube object

print("Current Calibration Information:")
print(f"\nData shape: {dataset.data.shape}")
print(f"Data type: {dataset.data.dtype}")
print(f"Min value: {dataset.data.min()}")
print(f"Max value: {dataset.data.max()}")
print(f"Mean value: {dataset.data.mean():.2f}")

from py4DSTEM import show
dp = dataset.data[:,:,10,10]

#dataset.metadata


# 2. Call the swap_RQ method on the datacube object
dataset.swap_RQ()
print("Current Calibration Information:")
print(f"\nData shape: {dataset.data.shape}")
print(f"Data type: {dataset.data.dtype}")
print(f"Min value: {dataset.data.min()}")
print(f"Max value: {dataset.data.max()}")
print(f"Mean value: {dataset.data.mean():.2f}")

dp2 = dataset.data[:,:,10,10]

#dataset.get_virtual_diffraction_shifts_center()

#dataset.calibration['Q_pixel_size'] = 2
#dataset.calibration['set_step_size_nm'] = step_size
# Example manual calibration
# Assuming your dataset object is named 'dataset'
dataset.calibration.R_pixel_size = 0.1
dataset.calibration.R_pixel_units = 'nm'

# 2. Set the Reciprocal-space (Diffraction) calibration
dataset.calibration.Q_pixel_size = 1
dataset.calibration.Q_pixel_units = 'A^-1'

# 3. Apply changes (important for some internal py4dstem functions)
print(dataset.calibration)
#dataset.get_diffraction_shifts()

show(dp)
show(dp2)

Current Calibration Information:

Data shape: (480, 448, 77, 17)
Data type: uint16
Min value: 195
Max value: 38473
Mean value: 253.63
Current Calibration Information:

Data shape: (77, 17, 480, 448)
Data type: uint16
Min value: 195
Max value: 38473
Mean value: 253.63
Calibration( A Metadata instance called 'calibration', containing the following fields:

             Q_pixel_size:    1
             R_pixel_size:    0.1
             Q_pixel_units:   A^-1
             R_pixel_units:   nm
             QR_flip:         False
)


AttributeError: module 'py4DSTEM.process' has no attribute 'alignment'

## Task 3: Virtual Detector Reconstruction

Generate a Virtual Bright Field (BF) image by integrating the central transmitted disk.

Generate an Annular Dark Field (ADF) image by integrating the scattered electrons in an outer ring.

Compare the Z-contrast in the ADF image to the diffraction contrast in the BF image.

In [ ]:
# Your code here

## Task 4: Basic 4D-STEM Visualization

Launch the interactive 4D-STEM browser using dataset.show() (if using a local GUI) or py4D.show_image().

Export a publication-quality figure of a virtual ADF image with a scale bar and a perceptually uniform colormap (e.g., magma).

In [2]:
# Your code here




from matplotlib.patches import Rectangle
if scale_bar_pixels < adf_image.shape[1] - 5:
    # Position scale bar in bottom-left corner
    bar_x, bar_y = 2, adf_image.shape[0] - 4
    scale_bar = Rectangle((bar_x, bar_y), scale_bar_pixels, 1, fill=True, color='white', linewidth=1)
    ax.add_patch(scale_bar)
    ax.text(bar_x + scale_bar_pixels/2, bar_y - 1, f'{scale_bar_length} nm', ha='center', va='top', 
            color='white', fontsize=10, fontweight='bold')

# Add colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label('Integrated Intensity (a.u.)', fontsize=12)

plt.tight_layout()
plt.savefig('virtual_adf_figure_sisige.png', dpi=300, bbox_inches='tight')
plt.show()

## Task 5: Finalize and Submit

Update your README.md with a brief explanation of how virtual detectors allow post-acquisition imaging.

Push the completed Week 02 notebook to your GitHub repository.

Submit the repository link on Canvas.

In [ ]:
# Your code here